In [3]:
import numpy as np
import pandas as pd
import pickle
import torch

In [4]:
with open('/Users/jihwanlim/Desktop/full_prediction/int2cell.pickle', 'rb') as handle:
    int2cell = pickle.load(handle)

with open('/Users/jihwanlim/Desktop/full_prediction/int2gene.pickle', 'rb') as handle:
    int2gene = pickle.load(handle)

def sort_genes_by_value(row):
    return row.sort_values(ascending=False).index.tolist()

crispr_neurobl = pd.read_csv('/Users/jihwanlim/Desktop/masterdissertation/data/crispr_Neuroblastoma_Reactome.csv', index_col=0)
heterodata_obj = torch.load('/Users/jihwanlim/Desktop/heteroData_gene_cell_Neuroblastoma_Reactome_crispr-1_5_cgp_cnv.pt')

dep_genes = list(set(heterodata_obj['gene', 'dependency_of', 'cell'].edge_index[0].numpy()))
dep_genes_str = []

for gene_id in dep_genes:
    dep_genes_str.append(int2gene[gene_id])

crispr_neurobl_filtered = crispr_neurobl[dep_genes_str]
crispr_neurobl_bin = crispr_neurobl_filtered.applymap(lambda x: int(x < -1.5))

def avg_df(BASE='/Users/jihwanlim/Desktop/Final_Result/Files', gene_feat='cgp',cell_feat='cellempty',top = 0.05):

    df1 = pd.read_csv(f'{BASE}/Neuroblastoma_Reactome_noRPL_STD_crispr-1_5_HetGNN_{gene_feat}_{cell_feat}_1.csv', index_col=0, header=0)
    df42 = pd.read_csv(f'{BASE}/Neuroblastoma_Reactome_noRPL_STD_crispr-1_5_HetGNN_{gene_feat}_{cell_feat}_42.csv', index_col=0, header=0)
    df999 = pd.read_csv(f'{BASE}/Neuroblastoma_Reactome_noRPL_STD_crispr-1_5_HetGNN_{gene_feat}_{cell_feat}_999.csv', index_col=0, header=0)

    df1.columns = [int(col) for col in df1.columns]
    df42.columns = [int(col) for col in df42.columns]
    df999.columns = [int(col) for col in df999.columns]

    df_avg = (df1+df42+df999)/3
    df_avg.rename(columns=int2gene, index=int2cell, inplace=True)

    New_prior = df_avg.apply(sort_genes_by_value, axis=1)
    sorted_genes = pd.DataFrame(New_prior.tolist(), index=df_avg.index)
    top_gene_num = int(len(df_avg.columns) * top)
    print(int(len(df_avg.columns) * top))
    df_selected = sorted_genes.iloc[: , :top_gene_num]

    gene_counts = pd.Series(df_selected.values.ravel()).value_counts()
    print(len(gene_counts))
    final_df = pd.DataFrame(gene_counts, columns=[f'times in top {int(top*100)}%'])
    
    final_df['Mean Prob'] = 0
    final_df['Original Dependency'] = 0
    final_df['Original Dependency Score'] = 0

    for gene in final_df.index:
        coordinates = np.where(df_selected == gene)
        cell_lines_with_gene = df_selected.apply(lambda row: gene in row.values, axis=1)

        values = [df_avg[gene][int2cell[cell_id]] for cell_id in coordinates[0]]

        mean_values = np.mean(values)

        final_df['Mean Prob'][gene] = mean_values

        if gene in dep_genes_str:
            final_df['Original Dependency'][gene] = crispr_neurobl_bin[gene].sum()
            final_df['Original Dependency Score'][gene] = crispr_neurobl_filtered.loc[cell_lines_with_gene, gene].mean()

    #final_df = final_df.sort_values(by='real_dep')
    final_df = final_df.sort_values(by=f'times in top {int(top*100)}%', ascending=False )
    return final_df
    # final_df.to_csv(f'/Users/jihwanlim/Desktop/Final_Result/final_{gene_feat}_{cell_feat}.csv')
    

In [ ]:
# Scores for all cancer cell lines
crispr_neurobl['TELO2']

In [ ]:
# Scores for Neuroblastoma cancer cell lines
crispr_neurobl_filtered['TELO2']

In [43]:
cgp_empty = avg_df()
go_lin_empty = avg_df(gene_feat='golin')

701
1000


/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency'][gene] = crispr_neurobl_bin[gene].sum()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency Score'][gene] = crispr_neurobl_filtered.loc[cell_lines_with_gene, gene].mean()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

701
1002


/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency'][gene] = crispr_neurobl_bin[gene].sum()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency Score'][gene] = crispr_neurobl_filtered.loc[cell_lines_with_gene, gene].mean()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_52793/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [44]:
cgp_empty.to_csv(f'/Users/jihwanlim/Desktop/Final_Result/HetGNN-Base_CGPtop5%.csv')
go_lin_empty.to_csv(f'/Users/jihwanlim/Desktop/Final_Result/HetGNN-Lin_GOtop5%.csv')

In [37]:
cgp_ = set(cgp_empty[(cgp_empty['times in top 5%'] >= 35) &(cgp_empty['Mean Prob'] > 0.8)& (cgp_empty['Original Dependency Score'] > -1.5) &(cgp_empty['Original Dependency'] < 10)].index.tolist())
go_lin_ = set(go_lin_empty[(go_lin_empty['times in top 5%'] >= 35) &(go_lin_empty['Mean Prob'] > 0.8)& (go_lin_empty['Original Dependency Score'] > -1.5) &(go_lin_empty['Original Dependency'] < 10)].index.tolist())
cgp_.intersection(go_lin_)

{'ALYREF',
 'ARCN1',
 'ARIH1',
 'ATP6V1F',
 'ATR',
 'BCL2L1',
 'BOP1',
 'CLTC',
 'DCLRE1B',
 'DDX55',
 'DHX33',
 'DHX9',
 'EIF3E',
 'EIF4A1',
 'EIF5',
 'GARS1',
 'GGPS1',
 'GMPS',
 'GTF2E1',
 'HAUS6',
 'HSPA8',
 'INCENP',
 'NAPA',
 'NELFB',
 'NOB1',
 'NUP88',
 'NVL',
 'NXT1',
 'PABPC1',
 'PDCD11',
 'PLK4',
 'PMPCA',
 'PPAN',
 'PRPF4',
 'PSMC1',
 'RAD51',
 'RAD9A',
 'RNPS1',
 'ROMO1',
 'RPP40',
 'RRS1',
 'SCFD1',
 'SMC1A',
 'SMC2',
 'SRSF11',
 'TBCB',
 'TBL3',
 'TOPBP1',
 'TPT1',
 'TSR1',
 'USP7',
 'VMP1',
 'WDR18',
 'WDR55'}

In [11]:
spot_genes = ['ATR' , 'BCL2L1','CLTC' , 'PLK4' , 'RAD51' , 'RRS1' , 'SMC2' ,  'TOPBP1']

In [41]:
cgp_empty.loc[spot_genes,:].to_csv(f'/Users/jihwanlim/Desktop/Final_Result/HetGNN-Base_CGP_spot.csv')
go_lin_empty.loc[spot_genes,:].to_csv(f'/Users/jihwanlim/Desktop/Final_Result/HetGNN-Lin_GO_spot.csv')

In [5]:
cgp_zero = set(cgp_empty[(cgp_empty['Mean Prob'] > 0.1) & (cgp_empty['Original Dependency'] == 0)].index.tolist())
go_lin_zero = set(go_lin_empty[(go_lin_empty['Mean Prob'] > 0.1) &(go_lin_empty['Original Dependency'] == 0)].index.tolist())

In [7]:
go_lin_zero

set()

In [47]:
cgp_predicted = set(cgp_empty[(cgp_empty['times in top 5%'] == 37) & (cgp_empty['Original Dependency'] == 37)].index.tolist())
go_lin_predicted = set(go_lin_empty[(go_lin_empty['times in top 5%'] == 37) &(go_lin_empty['Original Dependency'] == 37)].index.tolist())
print(len(cgp_predicted),len(go_lin_predicted))

70 70


In [48]:
cgp_real = set(cgp_empty[ (cgp_empty['Original Dependency'] == 37)].index.tolist())
go_lin_real = set(go_lin_empty[(go_lin_empty['Original Dependency'] == 37)].index.tolist())
print(len(cgp_real),len(go_lin_real))

70 70


In [50]:
len(cgp_predicted.intersection(go_lin_real))

70

In [53]:
cgp_predicted_true = set(cgp_empty[(cgp_empty['times in top 5%'] == 37) &( cgp_empty['Mean Prob'] > 0.5) ].index.tolist())
go_lin_predicted_true= set(go_lin_empty[(go_lin_empty['times in top 5%'] == 37)& (go_lin_empty['Mean Prob'] > 0.5) ].index.tolist())
print(len(cgp_zero),len(go_lin_zero))

540 550


In [54]:
len(cgp_predicted_true.intersection(go_lin_predicted_true))

517

In [55]:
for i in cgp_predicted_true:
    if i not in dep_genes_str:
        print(i)

In [56]:
for i in go_lin_predicted_true:
    if i not in dep_genes_str:
        print(i)

In [60]:
cgp_empty = avg_df(top=1)
go_lin_empty = avg_df(gene_feat='golin',top=1)

14034
14034


/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency'][gene] = crispr_neurobl_bin[gene].sum()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency Score'][gene] = crispr_neurobl_filtered.loc[cell_lines_with_gene, gene].mean()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

14034
14034


/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency'][gene] = crispr_neurobl_bin[gene].sum()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Original Dependency Score'][gene] = crispr_neurobl_filtered.loc[cell_lines_with_gene, gene].mean()
/var/folders/9b/h4yb7lq93159q0tlmdjvm0780000gn/T/ipykernel_18948/2297484410.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [63]:
cgp_whole = set(cgp_empty[(cgp_empty['Mean Prob'] > 0.05) & (cgp_empty['Original Dependency'] == 0)].index.tolist())
go_lin_whole = set(go_lin_empty[(go_lin_empty['Mean Prob'] > 0.05) &(go_lin_empty['Original Dependency'] == 0)].index.tolist())
print(len(cgp_whole),len(go_lin_whole))

1 3


In [64]:
go_lin_whole

{'CHMP1A', 'PGGT1B', 'PSMG2'}

In [65]:
cgp_whole

{'LIPK'}

############NOT USED############################

In [68]:
train_cgp_1 = torch.load('/Users/jihwanlim/Desktop/traindata/cgp_1.pt')
train_cgp_42 = torch.load('/Users/jihwanlim/Desktop/traindata/cgp_42.pt')
train_cgp_999 = torch.load('/Users/jihwanlim/Desktop/traindata/cgp_99.pt')

In [12]:
gene2int = {value: key for key, value in int2gene.items()}

In [22]:
spot_genes_int = []
for i in spot_genes:
    spot_genes_int.append(gene2int[i])

In [51]:
spot_crispr_neurobl_bin = pd.DataFrame('-', index=crispr_neurobl_bin.index, columns=spot_genes)
spot_crispr_neurobl_bin

,ATR,BCL2L1,CLTC,PLK4,RAD51,RRS1,SMC2,TOPBP1
ModelID,,,,,,,,
ACH-000078,-,-,-,-,-,-,-,-
ACH-000099,-,-,-,-,-,-,-,-
ACH-000120,-,-,-,-,-,-,-,-
ACH-000149,-,-,-,-,-,-,-,-
ACH-000227,-,-,-,-,-,-,-,-
ACH-000259,-,-,-,-,-,-,-,-
ACH-000260,-,-,-,-,-,-,-,-
ACH-000310,-,-,-,-,-,-,-,-
ACH-000312,-,-,-,-,-,-,-,-


In [56]:
def cellline(df,spot_crispr_neurobl_bin_df):
    indices = []
    corresponding_values = []
    found_values = []
    corresponding_labels = []

    for value in spot_genes_int:
        # Check if the value is in edge_index_0
        found_indices = (df['gene', 'dependency_of', 'cell'].edge_label_index[0] == value).nonzero(as_tuple=True)[0]
        if found_indices.numel() > 0:  # If the tensor is not empty
            for index in found_indices:
                indices.append(index.item())
                corresponding_values.append(df['gene', 'dependency_of', 'cell'].edge_label_index[1][index].item())
                found_values.append(value)
                corresponding_labels.append(df['gene', 'dependency_of', 'cell'].edge_label[index].item())

    for value, index, corresponding_value, label in zip(found_values, indices, corresponding_values, corresponding_labels):

        if label == 1.0:
            spot_crispr_neurobl_bin_df.loc[int2cell[corresponding_value],int2gene[value]] = 1
        #     print(f"Gene {int2gene[value]} is dependent to {int2cell[corresponding_value]}")
        else:
            spot_crispr_neurobl_bin_df.loc[int2cell[corresponding_value],int2gene[value]] = 0
        #     print(f"Gene {int2gene[value]} is not dependent to {int2cell[corresponding_value]}")
    return spot_crispr_neurobl_bin_df


In [74]:
cgp42 = cellline(train_cgp_42,spot_crispr_neurobl_bin)
cgp1 = cellline(train_cgp_1,spot_crispr_neurobl_bin)
cgp999 = cellline(train_cgp_999,spot_crispr_neurobl_bin)

In [75]:
cgp1

,ATR,BCL2L1,CLTC,PLK4,RAD51,RRS1,SMC2,TOPBP1
ModelID,,,,,,,,
ACH-000078,-,1,-,-,-,1,-,-
ACH-000099,-,-,-,-,-,-,0,-
ACH-000120,-,-,1,-,1,-,-,-
ACH-000149,-,0,1,1,-,-,-,-
ACH-000227,1,-,-,-,1,-,-,-
ACH-000259,0,-,-,-,-,-,-,0
ACH-000260,1,-,-,1,0,-,-,1
ACH-000310,-,-,-,-,-,1,1,-
ACH-000312,-,-,-,-,-,-,-,1


In [76]:
cgp42

,ATR,BCL2L1,CLTC,PLK4,RAD51,RRS1,SMC2,TOPBP1
ModelID,,,,,,,,
ACH-000078,-,1,-,-,-,1,-,-
ACH-000099,-,-,-,-,-,-,0,-
ACH-000120,-,-,1,-,1,-,-,-
ACH-000149,-,0,1,1,-,-,-,-
ACH-000227,1,-,-,-,1,-,-,-
ACH-000259,0,-,-,-,-,-,-,0
ACH-000260,1,-,-,1,0,-,-,1
ACH-000310,-,-,-,-,-,1,1,-
ACH-000312,-,-,-,-,-,-,-,1


In [77]:
cgp999

,ATR,BCL2L1,CLTC,PLK4,RAD51,RRS1,SMC2,TOPBP1
ModelID,,,,,,,,
ACH-000078,-,1,-,-,-,1,-,-
ACH-000099,-,-,-,-,-,-,0,-
ACH-000120,-,-,1,-,1,-,-,-
ACH-000149,-,0,1,1,-,-,-,-
ACH-000227,1,-,-,-,1,-,-,-
ACH-000259,0,-,-,-,-,-,-,0
ACH-000260,1,-,-,1,0,-,-,1
ACH-000310,-,-,-,-,-,1,1,-
ACH-000312,-,-,-,-,-,-,-,1
